# Aspect-based summarization: pair classification and clustering

## Imports

In [1]:
!pip3 install nltk stanza transformers evaluate seqeval --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.5/802.5 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.7/353.7 kB 33.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from abc import ABC, abstractmethod
from collections import defaultdict, Counter
from copy import deepcopy
from google.colab import drive
from ast import literal_eval
import logging
import os
import random
from string import punctuation
from typing import Union
from functools import reduce

import numpy as np
import pandas as pd

import stanza
import nltk
from nltk import collocations

import torch

import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import pipeline

from sklearn.cluster import AffinityPropagation
from scipy.spatial import distance

from sklearn.metrics import classification_report

In [ ]:
PUNCTUATION = punctuation.replace('\'', '').replace('"', '')

In [ ]:
stanza.download('ru')
nltk.download('popular')

INFO:stanza:Downloading default packages for language: ru (Russian) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip

True

In [ ]:
RUS_SW = set(nltk.corpus.stopwords.words('russian') + ['минус', 'плюс', 'отзыв', 'достоинство', 'недостаток', 'комментарий', 'это'])

In [ ]:
def seed_everything(seed=42) -> None:
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False

In [ ]:
seed_everything()

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
SENTIMENT = ['neutral', 'positive', 'negative']
id2label_sentiment = {i: label for i, label in enumerate(SENTIMENT)}
label2id_sentiment = {v: k for k, v in id2label_sentiment.items()}

In [ ]:
SPECIAL_TOKENS = {
    'bert': {
        'cls': '[CLS]',
        'sep': '[SEP]'
    },
    'xlm': {
        'cls': '<s>',
        'sep': '</s>'
    }
}

## Data

In [ ]:
laptop_reviews = pd.read_csv('/content/drive/MyDrive/Summarization/laptop data/reviews.tsv', delimiter='\t')
laptop_aspects = pd.read_excel('/content/drive/MyDrive/Summarization/laptop data/datasets/aspects-surface.xlsx')

In [ ]:
laptop_aspects['term_from'].astype(int)
laptop_aspects['term_to'].astype(int);

In [ ]:
laptop_reviews.head()

,id,product_id,text,pluses,minuses,review,stars
0,0,0,Плюсы: Unix Трекпад Качественный экран Качеств...,Unix Трекпад Качественный экран Качество сборк...,Трекпад начал пощелкивать спустя 3 месяца посл...,Я работаю разработчиком и покупал ноутбук имен...,5
1,1,0,Плюсы: - качество картинки на мониторе - тачпа...,- качество картинки на мониторе - тачпад прост...,"- корпус хрупковат, чуть ударил - вмятина на а...","Успел купить пару месяцев назад, сейчас смотрю...",5
2,2,0,"Плюсы: алюминий, марка, батарейка, экран, вес,...","алюминий, марка, батарейка, экран, вес, звук",софт ооооооооочень дорогой и многое нет!,"купил, первое впечатление ВАУ . Потом когда до...",4
3,3,0,Плюсы: - Экран - Тачпад - Качество сборки - Фи...,- Экран - Тачпад - Качество сборки - Фишки Mac...,"- Отсутствие Ethernet порта, все-таки ethernet...",Отличный ноутбук. После работы за таким экрано...,5
4,4,0,Плюсы: + вес + производительность + Дизайн + у...,+ вес + производительность + Дизайн + удобная ...,- со временем появляются битые пиксели - кабел...,Пользуюсь моделью 2013 года уже 2.5 года. Моде...,5


In [ ]:
laptop_reviews[laptop_reviews['product_id'] == 1]

,id,product_id,text,pluses,minuses,review,stars
10,10,1,Плюсы: Unix Трекпад Качественный экран Качеств...,Unix Трекпад Качественный экран Качество сборк...,Трекпад начал пощелкивать спустя 3 месяца посл...,Я работаю разработчиком и покупал ноутбук имен...,5
11,11,1,Плюсы: - качество картинки на мониторе - тачпа...,- качество картинки на мониторе - тачпад прост...,"- корпус хрупковат, чуть ударил - вмятина на а...","Успел купить пару месяцев назад, сейчас смотрю...",5
12,12,1,"Плюсы: алюминий, марка, батарейка, экран, вес,...","алюминий, марка, батарейка, экран, вес, звук",софт ооооооооочень дорогой и многое нет!,"купил, первое впечатление ВАУ . Потом когда до...",4
13,13,1,Плюсы: - Экран - Тачпад - Качество сборки - Фи...,- Экран - Тачпад - Качество сборки - Фишки Mac...,"- Отсутствие Ethernet порта, все-таки ethernet...",Отличный ноутбук. После работы за таким экрано...,5
14,14,1,Плюсы: + вес + производительность + Дизайн + у...,+ вес + производительность + Дизайн + удобная ...,- со временем появляются битые пиксели - кабел...,Пользуюсь моделью 2013 года уже 2.5 года. Моде...,5
15,15,1,"Плюсы: Это apple, внем все на самом высше уров...","Это apple, внем все на самом высше уровне! Куп...",Цена. Цена. Цена. Но это наверное даже и +. Св...,Думаю ноут мне года на три точно. В поездках в...,5
16,16,1,"Плюсы: Дисплей -- если вы фотограф, то этот ди...","Дисплей -- если вы фотограф, то этот дисплей д...",После некоторого времени использования при раб...,Конечно нужно сказать про цену. Она не маленьк...,5
17,17,1,Плюсы: Всё! Трекпад Автономность (лучшая среди...,Всё! Трекпад Автономность (лучшая среди всех н...,"Слабый корпус, многие пишут что повреждают его...","Это лучший представитель рабочих ноутбуков, об...",5
18,18,1,Плюсы: Все нравится. Дико рад пользованию данн...,Все нравится. Дико рад пользованию данным комп...,Таких нет.,Я на 2 года забыл - что такое загрузка и подви...,5
19,19,1,Плюсы: + Дизайн (тонкий и красивый) + Вес устр...,+ Дизайн (тонкий и красивый) + Вес устройства ...,- Цена (почти полностью оправдана) - Софт (под...,Плюсы точно перевешивают,5


## Collocations

We can get collocations in two ways:
* Get collocations from each review
* Get collocations from the set of reviews by product

Let us get collocations from each review because firstly we should extract all of the aspects.

In [ ]:
nlp = stanza.Pipeline('ru', proccessors='tokenize,pos,lemma', verbose=False, use_gpu=True)

In [ ]:
bigram_measures = collocations.BigramAssocMeasures()
trigram_measures = collocations.TrigramAssocMeasures()

In [ ]:
def get_text_info(text: str) -> list:
    '''
    Get info about tokens from text with stanza.
    '''
    doc = nlp(text)
    lemmas = []  # verified lemmas
    lemmas_tokens = []  # verified tokens with info
    tokens = []  # all tokens
    sents_with_end = {}  # sentences 
    for sent in doc.sentences:
        for word, token in zip(sent.words, sent.tokens):
            tokens.append((token.text, token.start_char, token.end_char))
            lemma = word.lemma.lower()
            if lemma not in RUS_SW and lemma not in PUNCTUATION:
                lemmas.append(lemma)
                lemmas_tokens.append((token.text, token.start_char, token.end_char))
            end_idx = token.end_char  # last token in the current sentence
            sents_with_end[end_idx] = sent.text

    logging.warning('Got text info...')

    return lemmas, lemmas_tokens, tokens, sents_with_end

In [ ]:
def get_collocations(lemmatized: list, n: int = 10) -> list:
    '''
    Get list of collocations from the text.
    '''
    unigrams = [lemma for (lemma, count) in Counter(lemmatized).most_common(n)]

    bigram_finder = collocations.BigramCollocationFinder.from_words(lemmatized)
    bigrams = bigram_finder.nbest(bigram_measures.pmi, n)

    trigram_finder = collocations.TrigramCollocationFinder.from_words(lemmatized)
    trigrams = trigram_finder.nbest(trigram_measures.pmi, n)

    logging.warning('Got collocations...')

    return unigrams, bigrams, trigrams

In [ ]:
def get_aspects_from_collocations(collocations: list, tokenized: list, text: str) -> list:
    '''
    Get aspects from tokens by lemmatized collocations.
    '''
    unigrams, bigrams, trigrams = collocations
    lemmatized, lemmas_tokens, tokens, sents_with_end = tokenized

    aspects = []

    # unigram aspects
    for unigram in unigrams:
        for lemma, token in zip(lemmatized, lemmas_tokens):
            if unigram == lemma:  # search all aspects with this lemma
                aspects.append(token)

    # n-grams aspects
    ngrams = bigrams + trigrams

    for ngram in ngrams:
        for start_idx, start_lemma in enumerate(lemmatized):
            if ngram[0] == start_lemma:
                for i in range(7):  # search in the right window of 7 tokens
                    if start_idx+i < len(lemmatized):
                        if ngram[-1] == lemmatized[start_idx+i]:
                            token_1, token_2 = lemmas_tokens[start_idx], lemmas_tokens[start_idx+i]
                            ngram_tokens = tokens[tokens.index(token_1):tokens.index(token_2)+1]
                            ngram_start = ngram_tokens[0][1]
                            ngram_end = ngram_tokens[-1][2]
                            full_ngram = text[ngram_start:ngram_end]
                            ngram_tuple = (full_ngram, ngram_start, ngram_end)
                            if ngram_tuple not in aspects:
                                aspects.append(ngram_tuple)

    aspects.sort(key=lambda x: x[1])

    logging.warning('Got aspects...')

    return aspects

In [ ]:
products_aspects = []
for i in range(1, 8):
    product_texts = laptop_reviews[laptop_reviews['product_id'] == i]['text'].values.tolist()
    products_aspects.append(product_texts)
    # print(len(product_texts))

len(products_aspects)

7

In [ ]:
products_aspects[0][:3]

['Плюсы: Unix Трекпад Качественный экран Качество сборки Долгое время работы от аккумулятора Отличный звук от колонок Минусы: Трекпад начал пощелкивать спустя 3 месяца после покупки Аллюминиевый корпус вминается, если уронить/ударить ноутбук  Отзыв: Я работаю разработчиком и покупал ноутбук именно для работы. Пользуюсь ноутбуком уже 5-й месяц. До этого пару лет пользовался Thinkpad x220 и Linux. Очень радует то, что операционная система семейства Unix, а также то, что в интернете огромное количество информации под macOS и OS X для разработчиков, когда нужно что-то скомпилировать/собрать и т.д. Это как Linux, только очень качественный и доведенный до ума без его болячек, вроде того, что что-нибудь отвалится в процессе работы само по себе и т.д. Из недостатков: - корпус можно было бы сделать из более прочного материала - я один раз уронил ноутбук на кафель и получил неплохую вмятину на крышке. - не знаю с чем это связано, но трекпад начал немного потрескивать при касании в верхнем правом

In [ ]:
gold_aspects = laptop_aspects[laptop_aspects['text_id'] == 11][['term', 'term_from', 'term_to']].values.tolist()

gold_aspects

[['монитор', 30, 38],
 ['тачпад', 41, 47],
 ['ssd', 78, 81],
 ['экрана', 110, 116],
 ['клавиатуры', 129, 139],
 ['разъемы USB, HDMI и отверстие для SD карточек', 153, 198],
 ['аккумулятор', 358, 369],
 ['операционная система', 389, 409],
 ['вентиляторов', 513, 525],
 ['экрана', 536, 542],
 ['можно класть ноутбук на любой бок', 545, 578],
 ['корпус', 589, 595],
 ['чуть ударил - вмятина на алюминии', 607, 640],
 ['экран', 643, 648],
 ['экране', 724, 730],
 ['подвывает вентиляторами под нагрузкой', 760, 797],
 ['вентиляторами', 770, 783],
 ['макбуке', 1070, 1077],
 ['есть самые нужные разъемы и наворотики', 1157, 1195],
 ['разъемы', 1175, 1182],
 ['наворотики', 1185, 1195],
 ['датчика света', 1201, 1214],
 ['ноутбук', 1369, 1376],
 ['Экран', 1395, 1400],
 ['монитора', 1444, 1452],
 ['экране', 1494, 1500],
 ['Тачпад', 1556, 1562],
 ['Аккумулятор', 1686, 1697],
 ['машина', 1741, 1747],
 ['Корпус', 1811, 1817],
 ['ноутбук', 1924, 1931],
 ['корпус', 1971, 1977],
 ['алюминий', 2037, 2045],
 ['

In [ ]:
laptop_aspects.head()

,id,text_id,category,sent_from,sent_to,sentiment,sent,sent_term,normalized_sent_term,term_from,term_to,term,normalized_term,type
0,0,10,Non-performance,12,19,positive,Трекпад,Трекпад,Трекпад,12,19,Трекпад,Трекпад,explicit
1,1,10,Non-performance,20,32,positive,Качественный,Качественный экран,Качественный экран,33,38,экран,экран,explicit
2,2,10,Appearance,48,54,neutral,сборки,сборки,сборки,48,54,сборки,сборка,explicit
3,3,10,Performance,78,90,positive,аккумулятора,аккумулятора,аккумулятора,78,90,аккумулятора,аккумулятор,explicit
4,4,10,Non-performance,108,115,positive,колонок,колонок,колонки,108,115,колонок,колонки,explicit


In [ ]:
products_aspects = []
products_sentences = []
products_gold_sentiment = []

reviews_exact_match = 0
reviews_partial_match = 0
for i in range(1, 8):
    product_texts = laptop_reviews[laptop_reviews['product_id'] == i][['id', 'text']].values.tolist()
    
    product_aspects = []
    product_sentences = []
    product_gold_sentiment = []
    for idx, text in product_texts:
        # get_collocations
        tokenized = get_text_info(text)
        lemmas = tokenized[0]
        colls = get_collocations(lemmas, n=5)
        predicted_aspects = get_aspects_from_collocations(colls, tokenized, text)

        # print(predicted_aspects)

        # evaluation
        gold_aspects = laptop_aspects[laptop_aspects['text_id'] == idx][['term', 'term_from', 'term_to']].values.tolist()
        # print(gold_aspects)
        gold_sentiment = laptop_aspects[laptop_aspects['text_id'] == idx]['sentiment'].values.tolist()

        exact_match = len([aspect for aspect in predicted_aspects if list(aspect) in gold_aspects]) / len(predicted_aspects)

        partial = []
        sentiment = [None for _ in range(len(predicted_aspects))]
        # print(len(sentiment))
        for asp_idx, (aspect_1, start_1, end_1) in enumerate(predicted_aspects):
            for (aspect_2, start_2, end_2), sent in zip(gold_aspects, gold_sentiment):
                if (start_1 in range(start_2, end_2+1)) or \
                (end_1 in range(start_2, end_2+1)) or \
                (start_2 in range(start_1, end_1+1)) or \
                (end_2 in range(start_1, end_1+1)):
                    if aspect_1 not in partial:
                        partial.append(aspect_1)
                        sentiment[asp_idx] = sent

        partial_match = len(partial) / len(predicted_aspects)

        # print('Exact match', exact_match)
        reviews_exact_match += exact_match
        # print('Partial match', partial_match)
        reviews_partial_match += partial_match

        # store information
        product_aspects.append(predicted_aspects)
        product_sentences.append(tokenized[3])
        product_gold_sentiment.append(sentiment)

    products_aspects.append(product_aspects)
    products_sentences.append(product_sentences)
    products_gold_sentiment.append(product_gold_sentiment)

In [ ]:
print('Exact match:', reviews_exact_match / 70)
print('Partial match:', reviews_partial_match / 70)

Exact match: 0.10525628599994524
Partial match: 0.3785704569237695


In [ ]:
products_aspects[0][2]

[('софт', 60, 64),
 ('многое', 89, 95),
 ('ВАУ', 134, 137),
 ('ВАУ', 206, 209),
 ('Софт', 212, 216),
 ('весь', 217, 221),
 ('стоит', 234, 239),
 ('весь', 272, 276),
 ('многое', 287, 293),
 ('dropbox нет в appstore', 320, 342),
 ('dropbox нет в appstore . Родной', 320, 351),
 ('appstore . Родной', 334, 351),
 ('appstore . Родной quicktime', 334, 361),
 ('player это вообще', 362, 379),
 ('player это вообще тупость', 362, 387),
 ('Elmedia на ура', 468, 482),
 ('Elmedia на ура. Туповатое', 468, 493),
 ('стоит', 535, 540),
 ('mail облако', 575, 586),
 ('mail облако, так это отдельно', 575, 604)]

In [ ]:
print(len(products_aspects))
print(len(products_aspects[0]))

7
10


In [ ]:
print(len(products_sentences))
print(len(products_sentences[0]))

7
10


## Sentiment analysis

In [ ]:
def get_data(aspects: list, sents_with_end: dict, model: str) -> tuple:
    '''
    Get data for pair classification with stanza.
    '''
    spec_tokens = SPECIAL_TOKENS.get(model, None)
    if not spec_tokens:
        raise ValueError('Provide model name to get right special tokens!')

    sep = spec_tokens.get('sep', None)
    cls = spec_tokens.get('cls', None)

    data = []

    for asp in aspects:
        mention_end_char = asp[2]

        for end_char in sents_with_end:
            if end_char >= mention_end_char:
                # first sentence where end > end of mention
                sentence = sents_with_end[end_char]
                mention = asp[0]
                data.append(f'{cls} {sentence} {sep} {mention} {sep}')

                break  # one sentence only

    return data

In [ ]:
products_bert_data = []
for product_aspects, product_sentences in zip(products_aspects, products_sentences):
    product_data = []
    for aspects, sentences in zip(product_aspects, product_sentences):
        text_data = get_data(aspects, sentences, model='bert')
        product_data.append(text_data)
    products_bert_data.append(product_data)

In [ ]:
products_bert_data[0][0][:5]

['[CLS] Плюсы: Unix Трекпад Качественный экран Качество сборки Долгое время работы от аккумулятора Отличный звук от колонок Минусы: [SEP] Unix [SEP]',
 '[CLS] Плюсы: Unix Трекпад Качественный экран Качество сборки Долгое время работы от аккумулятора Отличный звук от колонок Минусы: [SEP] Трекпад [SEP]',
 '[CLS] Плюсы: Unix Трекпад Качественный экран Качество сборки Долгое время работы от аккумулятора Отличный звук от колонок Минусы: [SEP] Качественный [SEP]',
 '[CLS] Плюсы: Unix Трекпад Качественный экран Качество сборки Долгое время работы от аккумулятора Отличный звук от колонок Минусы: [SEP] работы [SEP]',
 '[CLS] Плюсы: Unix Трекпад Качественный экран Качество сборки Долгое время работы от аккумулятора Отличный звук от колонок Минусы: [SEP] аккумулятора Отличный [SEP]']

In [ ]:
products_xlmroberta_data = []
for product_aspects, product_sentences in zip(products_aspects, products_sentences):
    product_data = []
    for aspects, sentences in zip(product_aspects, product_sentences):
        text_data = get_data(aspects, sentences, model='xlm')
        product_data.append(text_data)
    products_xlmroberta_data.append(product_data)

In [ ]:
products_xlmroberta_data[0][0][:5]

['<s> Плюсы: Unix Трекпад Качественный экран Качество сборки Долгое время работы от аккумулятора Отличный звук от колонок Минусы: </s> Unix </s>',
 '<s> Плюсы: Unix Трекпад Качественный экран Качество сборки Долгое время работы от аккумулятора Отличный звук от колонок Минусы: </s> Трекпад </s>',
 '<s> Плюсы: Unix Трекпад Качественный экран Качество сборки Долгое время работы от аккумулятора Отличный звук от колонок Минусы: </s> Качественный </s>',
 '<s> Плюсы: Unix Трекпад Качественный экран Качество сборки Долгое время работы от аккумулятора Отличный звук от колонок Минусы: </s> работы </s>',
 '<s> Плюсы: Unix Трекпад Качественный экран Качество сборки Долгое время работы от аккумулятора Отличный звук от колонок Минусы: </s> аккумулятора Отличный </s>']

In [ ]:
rubert_model_checkpoint = '/content/drive/MyDrive/models/rubert-sentiment-nli_both'
mbert_model_checkpoint = '/content/drive/MyDrive/models/mbert-sentiment-nli_both'
xlmroberta_model_checkpoint = '/content/drive/MyDrive/models/xlmroberta-sentiment-nli_both'
rubert_tokenizer_checkpoint = 'ai-forever/ruBert-base'
mbert_tokenizer_checkpoint = 'bert-base-multilingual-cased'
xlmroberta_tokenizer_checkpoint = 'xlm-roberta-base'

In [ ]:
def load_model_and_tokenizer(model_path: str, tokenizer_path: str) -> tuple:
    '''
    Load model and tokenizer from paths.
    '''
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    return model, tokenizer

In [ ]:
def get_nli_sentiment(model_name: str, model: AutoModelForSequenceClassification, tokenizer: AutoTokenizer, data: list) -> list:
    '''
    Predict sentiment of aspect using transformer-based model.
    '''
    predicted_sentiment = []
    aspects = []
    # embeddings = []
    for text in data:
        tokenized = tokenizer(text, return_tensors='pt', truncation=True, max_length=100, padding=True, add_special_tokens=False)
        sep_indexes = (tokenized['input_ids'] == tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS[model_name]['sep'])).nonzero(as_tuple=True)[1]
        tokenized = tokenized.to(device)
        with torch.no_grad():
            output = model(**tokenized, output_hidden_states=True)
            classification_logits = output.logits
            # embeddings.append(hidden_states)
            embeddings = output.hidden_states[0][0]
            assert len(embeddings) == len(tokenized['input_ids'][0])
            if len(sep_indexes) == 2:
                aspect_embeddings = embeddings[sep_indexes[0]+1:sep_indexes[1]]
                aspect_embedding = aspect_embeddings.sum(axis=0) / len(aspect_embeddings)
                aspects.append(aspect_embedding.numpy())
        predicted_class_id = classification_logits.to('cpu').argmax().item()
        predicted_sentiment.append(model.config.id2label[predicted_class_id])

    return predicted_sentiment, aspects

### ruBERT

In [ ]:
model, tokenizer = load_model_and_tokenizer(rubert_model_checkpoint, rubert_tokenizer_checkpoint)

In [ ]:
rubert_products_sentiments = []
rubert_products_embeddings = []

sentiment_references = []
sentiment_predictions = []
for product_data, product_gold_sentiment in zip(products_bert_data, products_gold_sentiment):
    product_sentiment = []
    product_embeddings = []
    for text_data, gold_sentiment in zip(product_data, product_gold_sentiment):
        predicted_sentiment, aspects_embeddings = get_nli_sentiment('bert', model, tokenizer, text_data)
        product_sentiment.append(predicted_sentiment)
        for pred_sent, gold_sent in zip(predicted_sentiment, gold_sentiment):
            if pred_sent and gold_sent:
                sentiment_references.append(gold_sent)
                sentiment_predictions.append(pred_sent)
        product_embeddings.append(aspects_embeddings)
    rubert_products_sentiments.append(product_sentiment)
    rubert_products_embeddings.append(product_embeddings)

In [ ]:
print(len(sentiment_references))
print(len(sentiment_predictions))

548
548


In [ ]:
for aspect, sentiment in zip(products_aspects[0][2], rubert_products_sentiments[0][2]):
    print(aspect[0], sentiment)

софт negative
многое negative
ВАУ positive
ВАУ negative
Софт negative
весь negative
стоит negative
весь negative
многое negative
dropbox нет в appstore negative
dropbox нет в appstore . Родной negative
appstore . Родной negative
appstore . Родной quicktime negative
player это вообще negative
player это вообще тупость negative
Elmedia на ура positive
Elmedia на ура. Туповатое negative
стоит neutral
mail облако neutral
mail облако, так это отдельно negative


In [ ]:
print(classification_report(sentiment_references, sentiment_predictions))

              precision    recall  f1-score   support

    negative       0.78      0.64      0.71       174
     neutral       0.22      0.36      0.28        74
    positive       0.81      0.76      0.79       300

    accuracy                           0.67       548
   macro avg       0.60      0.59      0.59       548
weighted avg       0.72      0.67      0.69       548



### mBERT

In [ ]:
model, tokenizer = load_model_and_tokenizer(mbert_model_checkpoint, mbert_tokenizer_checkpoint)

In [ ]:
mbert_products_sentiments = []
mbert_products_embeddings = []

sentiment_references = []
sentiment_predictions = []
for product_data, product_gold_sentiment in zip(products_bert_data, products_gold_sentiment):
    product_sentiment = []
    product_embeddings = []
    for text_data, gold_sentiment in zip(product_data, product_gold_sentiment):
        predicted_sentiment, aspects_embeddings = get_nli_sentiment('bert', model, tokenizer, text_data)
        product_sentiment.append(predicted_sentiment)
        for pred_sent, gold_sent in zip(predicted_sentiment, gold_sentiment):
            if pred_sent and gold_sent:
                sentiment_references.append(gold_sent)
                sentiment_predictions.append(pred_sent)
        product_embeddings.append(aspects_embeddings)
    mbert_products_sentiments.append(product_sentiment)
    mbert_products_embeddings.append(product_embeddings)

In [ ]:
for aspect, sentiment in zip(products_aspects[0][2], mbert_products_sentiments[0][2]):
    print(aspect[0], sentiment)

софт negative
многое negative
ВАУ neutral
ВАУ negative
Софт negative
весь negative
стоит negative
весь negative
многое negative
dropbox нет в appstore negative
dropbox нет в appstore . Родной negative
appstore . Родной negative
appstore . Родной quicktime negative
player это вообще negative
player это вообще тупость negative
Elmedia на ура negative
Elmedia на ура. Туповатое positive
стоит positive
mail облако neutral
mail облако, так это отдельно negative


In [ ]:
print(classification_report(sentiment_references, sentiment_predictions))

              precision    recall  f1-score   support

    negative       0.62      0.51      0.56       174
     neutral       0.26      0.41      0.32        74
    positive       0.79      0.76      0.77       300

    accuracy                           0.63       548
   macro avg       0.56      0.56      0.55       548
weighted avg       0.66      0.63      0.64       548



### XLM-RoBERTa

In [ ]:
model, tokenizer = load_model_and_tokenizer(xlmroberta_model_checkpoint, xlmroberta_tokenizer_checkpoint)

In [ ]:
xlmroberta_products_sentiments = []
xlmroberta_products_embeddings = []

sentiment_references = []
sentiment_predictions = []
for product_data, product_gold_sentiment in zip(products_xlmroberta_data, products_gold_sentiment):
    product_sentiment = []
    product_embeddings = []
    for text_data, gold_sentiment in zip(product_data, product_gold_sentiment):
        predicted_sentiment, aspects_embeddings = get_nli_sentiment('xlm', model, tokenizer, text_data)
        product_sentiment.append(predicted_sentiment)
        for pred_sent, gold_sent in zip(predicted_sentiment, gold_sentiment):
            if pred_sent and gold_sent:
                sentiment_references.append(gold_sent)
                sentiment_predictions.append(pred_sent)
        product_embeddings.append(aspects_embeddings)
    xlmroberta_products_sentiments.append(product_sentiment)
    xlmroberta_products_embeddings.append(product_embeddings)

In [ ]:
for aspect, sentiment in zip(products_aspects[0][2], xlmroberta_products_sentiments[0][2]):
    print(aspect[0], sentiment)

софт negative
многое negative
ВАУ neutral
ВАУ negative
Софт negative
весь negative
стоит negative
весь negative
многое negative
dropbox нет в appstore negative
dropbox нет в appstore . Родной negative
appstore . Родной negative
appstore . Родной quicktime negative
player это вообще negative
player это вообще тупость negative
Elmedia на ура positive
Elmedia на ура. Туповатое positive
стоит negative
mail облако neutral
mail облако, так это отдельно negative


In [ ]:
len(xlmroberta_products_embeddings[0][2])

20

In [ ]:
print(classification_report(sentiment_references, sentiment_predictions))

              precision    recall  f1-score   support

    negative       0.66      0.61      0.63       174
     neutral       0.26      0.27      0.27        74
    positive       0.80      0.83      0.81       300

    accuracy                           0.68       548
   macro avg       0.57      0.57      0.57       548
weighted avg       0.68      0.68      0.68       548



## Summarization

In [ ]:
laptop_summarization = pd.read_excel('/content/drive/MyDrive/Summarization/laptop data/datasets/summarization.xlsx')

laptop_summarization.head()

,id,product_id,term,sentiment
0,0,1,тачпад,positive
1,1,1,экран,positive
2,2,1,аккумулятор,positive
3,3,1,колонки,positive
4,4,1,сборка,positive


In [ ]:
class Clusterisator(ABC):

    def __init__(self, n_clusters=None, random_state=42):
        self.n_clusters = n_clusters
        self.random_state = random_state

    @abstractmethod
    def _clusterisation(self):
        pass

    def _get_aspects(self, embeddings: list):
        '''
        Identify aspects after clusterisation.
        '''
        
        labels, centers = self._clusterisation(embeddings)

        labels = list(labels)

        min_dist_states = {}

        for idx, (label, embedding) in enumerate(zip(labels, embeddings)):
            if label not in min_dist_states:
                min_dist_states[label] = {'min_dist': 1, 'min_idx': None, 'embedding': None}
            # get embedding with minimum cosine distance
            # from center of cluster
            dist = distance.cosine(embedding, centers[label])
            min_dist = min_dist_states[label].get('min_dist', None)
            if min_dist:
                if min_dist > dist:
                    if min_dist_states[label]['embedding'] is not None:
                        # return previous embedding with minimum distance into the list
                        emb_idx = min_dist_states[label]['min_idx']
                        emb = min_dist_states[label]['embedding']
                        embeddings[emb_idx] = emb

                    min_dist_states[label]['min_dist'] = dist
                    min_dist_states[label]['min_idx'] = idx
                    min_dist_states[label]['embedding'] = embedding

                    # remove main aspect and it's embedding from clusters
                    # leave other embeddings of clusters
                    # to get summarized polarity
                    embeddings[idx] = None

        return labels, embeddings

    def get_summarized_aspects(self, embeddings: list, mentions: list, sentiment: list):
        '''
        Summarize sentiment by aspect in the clusters.
        '''
        embeddings = deepcopy(embeddings)
        labels, embeddings = self._get_aspects(embeddings)

        labels = [i - 1 for i in deepcopy(labels)]

        aspects = [0 for _ in range(len(set(labels)))]
        other_sentiment = [Counter() for _ in range(len(set(labels)))]

        for idx, (label, embedding) in enumerate(zip(labels, embeddings)):

            # get main aspect
            if embedding is None:
                aspects[label] = mentions[idx]
            other_sentiment[label][sentiment[idx]] += 1

        sentiments = [sentiment.most_common()[0][0] for sentiment in other_sentiment]

        return aspects, sentiments

In [ ]:
class AffinityPropagationClusterisator(Clusterisator):

    def _clusterisation(self, embeddings):

        affp = AffinityPropagation(random_state=self.random_state, damping=0.7).fit(embeddings)
        labels = affp.labels_
        centers = affp.cluster_centers_

        return labels, centers

In [ ]:
affp_clusterisator = AffinityPropagationClusterisator()

In [ ]:
gold_summarization = laptop_summarization[laptop_summarization['product_id'] == 1]

gold_summarization.head()

,id,product_id,term,sentiment
0,0,1,тачпад,positive
1,1,1,экран,positive
2,2,1,аккумулятор,positive
3,3,1,колонки,positive
4,4,1,сборка,positive


In [ ]:
gold_summarization.groupby('sentiment')['term'].apply(lambda x: x.tolist()).to_dict()

{'negative': ['алюминий',
  'корпус',
  'вентиляторы',
  'Отсутствие Ethernet порта',
  'Отсутствие поддержки NTFS',
  'ремонтопригодность',
  '2 usb',
  'кулер',
  'цена',
  'тепловыделению'],
 'neutral': ['Клавиатура', 'наворотики'],
 'positive': ['тачпад',
  'экран',
  'аккумулятор',
  'колонки',
  'сборка',
  'ноутбук',
  'операционная система',
  'ssd',
  'разъемы',
  'датчика света',
  'можно класть ноутбук на любой бок',
  'есть самые нужные разъемы',
  'марка',
  'вес',
  'звук',
  'железо',
  'габаритах',
  'производительность',
  'Thunderbolt',
  'дизайн',
  'Эргономичность',
  'открывания одной рукой',
  'отклик']}

In [ ]:
def evaluate_summarization(summarization: list, gold_summarization: list) -> None:
    '''
    Evaluate summarization by exact and partial match.
    '''

    gold_summarization = [gold_coll.lower() for gold_coll in gold_summarization]
    exact_match = len([coll for coll in summarization if coll.lower() in gold_summarization])

    partial_match = []
    for coll in summarization:
        coll = coll.lower()
        for gold_coll in gold_summarization:
            if coll in gold_coll or gold_coll in coll:
                if coll not in partial_match:
                    partial_match.append(coll)
    partial_match = len(partial_match)

    return exact_match, partial_match

### ruBERT

In [ ]:
rubert_summarization = []

summarization_length = 0
exact_match = 0
partial_match = 0

for product_embeddings, product_aspects, product_sentiment, product_id in\
zip(rubert_products_embeddings, products_aspects, rubert_products_sentiments, range(1, 11)):

    predicted_embeddings = reduce(lambda x, y: x + y, product_embeddings)
    predicted_aspects = reduce(lambda x, y: x + y, product_aspects)
    predicted_mentions = [aspect[0] for aspect in predicted_aspects]
    predicted_sentiment = reduce(lambda x, y: x + y, product_sentiment)

    aspects, sentiments = affp_clusterisator.get_summarized_aspects(
        predicted_embeddings,
        predicted_mentions,
        predicted_sentiment
        )
    
    summarization = defaultdict(list)
    for aspect, sentiment in zip(aspects, sentiments):
        summarization[sentiment].append(aspect)

    g_sum = gold_summarization.groupby('sentiment')['term'].apply(lambda x: x.tolist()).to_dict()

    for sent in ['positive', 'negative', 'neutral']:
        gold_colls = g_sum.get(sent, None)
        colls = summarization.get(sent, None)
        if gold_colls and colls:
            summarization_length += len(colls)
            sent_exact_match, sent_partial_match = evaluate_summarization(colls, gold_colls)
            exact_match += sent_exact_match
            partial_match += sent_partial_match

    rubert_summarization.append(summarization)

In [ ]:
rubert_summarization[0]

defaultdict(list,
            {'positive': ['работы',
              'просто',
              'HDMI и отверстие',
              'нормальная',
              'Экран',
              'экране',
              'просто',
              'очень',
              '4 ядерная',
              'Elmedia на ура',
              'ноутбук',
              'Apple',
              'Cmd вместо',
              'ноутбук',
              'года',
              'Купил',
              'самое',
              'Дисплей',
              'hd мониторы',
              'Force Touch -- это причина',
              'всех',
              'Дико',
              'Трекпад'],
             'neutral': ['Unix',
              'ssd диск, и при этом он не ацки',
              'Трекпад',
              'Force Touch'],
             'negative': ['appstore . Родной quicktime',
              'player это вообще тупость',
              'Elmedia на ура. Туповатое',
              'iPhone прямо',
              'Ctrl, а точка и запятой',
              '2 us

In [ ]:
print('Exact match:', exact_match / summarization_length)
print('Partial match:', partial_match / summarization_length)

Exact match: 0.029411764705882353
Partial match: 0.0661764705882353


### mBERT

In [ ]:
mbert_summarization = []

summarization_length = 0
exact_match = 0
partial_match = 0

for product_embeddings, product_aspects, product_sentiment, product_id in\
zip(mbert_products_embeddings, products_aspects, mbert_products_sentiments, range(1, 11)):

    predicted_embeddings = reduce(lambda x, y: x + y, product_embeddings)
    predicted_aspects = reduce(lambda x, y: x + y, product_aspects)
    predicted_mentions = [aspect[0] for aspect in predicted_aspects]
    predicted_sentiment = reduce(lambda x, y: x + y, product_sentiment)

    aspects, sentiments = affp_clusterisator.get_summarized_aspects(
        predicted_embeddings,
        predicted_mentions,
        predicted_sentiment
        )
    
    summarization = defaultdict(list)
    for aspect, sentiment in zip(aspects, sentiments):
        summarization[sentiment].append(aspect)

    g_sum = gold_summarization.groupby('sentiment')['term'].apply(lambda x: x.tolist()).to_dict()

    for sent in ['positive', 'negative', 'neutral']:
        gold_colls = g_sum.get(sent, None)
        colls = summarization.get(sent, None)
        if gold_colls and colls:
            summarization_length += len(colls)
            sent_exact_match, sent_partial_match = evaluate_summarization(colls, gold_colls)
            exact_match += sent_exact_match
            partial_match += sent_partial_match

    mbert_summarization.append(summarization)

In [ ]:
mbert_summarization[0]

defaultdict(list,
            {'negative': ['Трекпад',
              'болячек, вроде того, что что-нибудь',
              'работе',
              'очень',
              'нормальная',
              'нормальным',
              'пока',
              'dropbox нет в appstore',
              'appstore . Родной',
              'player это вообще',
              'Apple',
              '7, буква',
              'Ноутбук',
              'производительности',
              '70к. Во-вторых SSD',
              '150 было бы оправданой',
              'P.S. На пятый'],
             'positive': ['Unix',
              'Очень',
              'экране',
              '4 ядерная',
              '4 ядерная 15',
              'Cmd вместо Ctrl',
              'проблема',
              '2 usb, для подключения',
              'покупки',
              '+. Своих сразу',
              '59к сверху',
              'Дисплей',
              '30 000 рублей',
              'ноутбуков',
              'работы'],
         

In [ ]:
print('Exact match:', exact_match / summarization_length)
print('Partial match:', partial_match / summarization_length)

Exact match: 0.015384615384615385
Partial match: 0.03076923076923077


### XLM-RoBERTa

In [ ]:
xlmroberta_summarization = []

summarization_length = 0
exact_match = 0
partial_match = 0

for product_embeddings, product_aspects, product_sentiment, product_id in\
zip(xlmroberta_products_embeddings, products_aspects, xlmroberta_products_sentiments, range(1, 11)):

    predicted_embeddings = reduce(lambda x, y: x + y, product_embeddings)
    predicted_aspects = reduce(lambda x, y: x + y, product_aspects)
    predicted_mentions = [aspect[0] for aspect in predicted_aspects]
    predicted_sentiment = reduce(lambda x, y: x + y, product_sentiment)

    aspects, sentiments = affp_clusterisator.get_summarized_aspects(
        predicted_embeddings,
        predicted_mentions,
        predicted_sentiment
        )
    
    summarization = defaultdict(list)
    for aspect, sentiment in zip(aspects, sentiments):
        summarization[sentiment].append(aspect)

    g_sum = gold_summarization.groupby('sentiment')['term'].apply(lambda x: x.tolist()).to_dict()

    for sent in ['positive', 'negative', 'neutral']:
        gold_colls = g_sum.get(sent, None)
        colls = summarization.get(sent, None)
        if gold_colls and colls:
            summarization_length += len(colls)
            sent_exact_match, sent_partial_match = evaluate_summarization(colls, gold_colls)
            exact_match += sent_exact_match
            partial_match += sent_partial_match

    xlmroberta_summarization.append(summarization)

In [ ]:
xlmroberta_summarization[0]

defaultdict(list,
            {'negative': ['Трекпад',
              'ноутбук',
              'appstore . Родной',
              'стоит',
              'ноутбук',
              'Mac',
              'Ctrl, а точка и запятой',
              'проблема',
              'временем',
              'ethernet нужен переходник',
              '70к. Во-вторых',
              '70к. Во-вторых SSD',
              'SSD впаяна, при выходе',
              'hd мониторы. Шум',
              'колонки',
              'дешевым подпружиненым',
              'думать до сих пор'],
             'positive': ['работы',
              'просто',
              'HDMI и отверстие для SD',
              'пока',
              'очень',
              '15 дюймовая',
              'просто',
              'ВАУ',
              'Cmd вместо',
              'ssd диск, и при этом он не ацки',
              'качественный',
              '150 было бы оправданой',
              'обычной',
              'среди',
              'запятая 

In [ ]:
print('Exact match:', exact_match / summarization_length)
print('Partial match:', partial_match / summarization_length)

Exact match: 0.01606425702811245
Partial match: 0.04417670682730924
